## Image Classification - Deep Learning

In [3]:
#r "nuget: Microsoft.ML, 2.0.1"
#r "nuget: Microsoft.ML.Vision, 2.0.1"
#r "nuget: Microsoft.ML.ImageAnalytics, 2.0.1"
#r "nuget: Microsoft.ML.TensorFlow.Redist"

using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Vision;

Installed Packages Microsoft.ML, 2.0.1 Microsoft.ML.ImageAnalytics, 2.0.1 Microsoft.ML.TensorFlow.Redist, 0.14.0 Microsoft.ML.Vision, 2.0.1

In [4]:
public class ImageInput
{
    public string ImagePath { get; set; }
    public string Label { get; set; }
}

public class ImagePrediction
{
    public string PredictedLabel { get; set; }
    public float[] Score { get; set; }
}


In [5]:
static IEnumerable<ImageInput> LoadImagesFromDirectory(string folder)
{
    foreach (var dir in Directory.GetDirectories(folder))
    {
        var label = Path.GetFileName(dir);

        foreach (var file in Directory.GetFiles(dir))
        {
            yield return new ImageInput
            {
                ImagePath = file,
                Label = label
            };
        }
    }
}


In [6]:
var mlContext = new MLContext(seed: 1);

//Load images
IDataView data = mlContext.Data.LoadFromEnumerable(LoadImagesFromDirectory(@"images"));

// Build pipeline
var pipeline =mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "LabelKey",inputColumnName: "Label")    
    .Append(mlContext.Transforms.LoadRawImageBytes(outputColumnName: "input",imageFolder: "",inputColumnName: nameof(ImageInput.ImagePath)))

    //TensorFlow model
    .Append(mlContext.MulticlassClassification.Trainers
        .ImageClassification(new ImageClassificationTrainer.Options
        {
            FeatureColumnName = "input",
            LabelColumnName = "LabelKey",
            Arch = ImageClassificationTrainer.Architecture.ResnetV2101,
            Epoch = 20,
            BatchSize = 10,
            LearningRate = 0.01f,
            MetricsCallback = (m) => Console.WriteLine(m),
            ValidationSet = null
        }))

    // Decode prediction
    .Append(mlContext.Transforms.Conversion
        .MapKeyToValue(
            outputColumnName: "PredictedLabel",
            inputColumnName: "PredictedLabel"));

// Train
var model = pipeline.Fit(data);

// Save model
//mlContext.Model.Save(model, data.Schema, "resnet-model.zip");

Saver not created because there are no variables in the graph to restore
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   1
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   2
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   3
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   4
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   5
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   6
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   7
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   8
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:   9
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  10
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  11
Phase: Bottleneck Computation, Dataset used:      Train, Image Index:  12
Phase: Bottleneck Computation, Dataset 

In [7]:
var predictor = mlContext.Model.CreatePredictionEngine<ImageInput, ImagePrediction>(model);

var prediction = predictor.Predict(new ImageInput
{
    ImagePath = @"test\images.jpg"
});

Console.WriteLine($"Prediction: {prediction.PredictedLabel}, Scores: {string.Join(",", prediction.Score.Where(s => s > 0.01))} ");



Prediction: Dog, Scores: 0.09577557,0.90422446 
